In [9]:
import cv2
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
import datetime

In [1]:
!if not exist "./files" mkdir files
# Download Face detection XML 
!curl -L -o ./files/haarcascade_frontalface_default.xml https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
# Download emotion trained data
!curl -L -o ./files/emotion_model.hdf5 https://mechasolution.vn/source/blog/AI-tutorial/Emotion_Recognition/emotion_model.hdf5

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  4  908k    4 41130    0     0   100k      0  0:00:09 --:--:--  0:00:09  100k
100  908k  100  908k    0     0  1704k      0 --:--:-- --:--:-- --:--:-- 1700k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0  878k    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  3  878k    3 34137    0     0  18623      0  0:00:48  0:00:01  0:00:47 18613
100  878k  100  878k    0     0   332k      0  0:00:02  0:00:02 --:--:--  332k


In [ ]:
#https://m.blog.naver.com/choi_s_h/222135352825

In [10]:
# Face detection XML load and trained model loading
face_detection = cv2.CascadeClassifier('files/haarcascade_frontalface_default.xml')
emotion_classifier = load_model('files/emotion_model.hdf5', compile=False)
EMOTIONS = ["Angry" ,"Disgusting","Fearful", "Happy", "Sad", "Surprising", "Neutral"]


# Video capture using webcam
# 여기에 삽입할 영상을 넣으면 됩니다!
camera = cv2.VideoCapture('C:/Users/KIMJINU/Desktop/Marketing/Project/source.mp4')

x_values = []
y_values = []
for i in range (0,20):
    t = len(x_values)
    x_values.append(t)
    y_values.append(0)

while True:
    # Capture image from camera
    ret, frame = camera.read()

    height, width = frame.shape[:2]
    
    frame = cv2.resize(frame,(1280, 720))
    
    # Convert color to gray scale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Face detection in frame
    faces = face_detection.detectMultiScale(gray,
                                            scaleFactor=1.1,
                                            minNeighbors=5,
                                            minSize=(150,150))
    
    # Create empty image
    window = np.zeros(())
    canvas = np.ones((740, 800, 3), dtype="uint8") * 255
    constant= cv2.copyMakeBorder(frame,10,10,10,10,cv2.BORDER_CONSTANT,value=[255,255,255])
    tmp = np.hstack((constant, canvas))
    
    #UI 칸
    cv2.rectangle(tmp, (1310, 10), (1890, 90), (0, 0, 0), 2)
    cv2.rectangle(tmp, (1310, 110), (1890, 410), (0, 0, 0), 2)
    cv2.rectangle(tmp, (1310, 430), (1890, 730), (0, 0, 0), -1)
    cv2.rectangle(tmp, (1920, 10), (2070, 90), (0, 0, 0), 2)
    cv2.rectangle(tmp, (1920, 110), (2070, 730), (0, 0, 0), 2)
   
    
    # Perform emotion recognition only when face is detected
    if len(faces) > 0:
        
        # For the largest image
        face = sorted(faces, reverse=True, key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))[0]
        (fX, fY, fW, fH) = face
        
        # Resize the image to 48x48 for neural network
        roi = gray[fY:fY + fH, fX:fX + fW]
        roi = cv2.resize(roi, (48, 48))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
        
        # Emotion predict
        preds = emotion_classifier.predict(roi)[0]
        emotion_probability = np.max(preds)
        label = EMOTIONS[preds.argmax()]
        
        # Assign labeling
        cv2.putText(tmp, label, (fX, fY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
        cv2.rectangle(tmp, (fX, fY), (fX + fW, fY + fH), (0, 0, 255), 2)
        
            
        # Label printing
        for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
            #time
            time = str(datetime.datetime.now())
            cv2.putText(tmp, time[:22], (1380, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

            text = "{}: {:.2f}%".format(emotion, prob * 100)    
            w = int(prob * 300)
            
            if prob > 0.3:
                color = (0,0, 255)
            else:
                color = (0,255,0)
                
            cv2.rectangle(tmp, (1320, (i * 40) + 120), (w + 1320, (i * 40) + 160), (0,255,255), -1)
            cv2.putText(tmp, text, (1323, (i * 40) + 145), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
                
            if emotion == "Sad":
                text = "{}: {:.2f}%".format(emotion, prob * 100)    
                w = int(prob * 300)
            
                if prob > 0.5:
                    color_w = (0,0, 255)
                    label = "danger"
                else:
                    color_w = (0,255,0)
                    label = "safe"
            
                cv2.rectangle(tmp, (1620, 120), (w + 1620, 400), (0,0,255), -1)
                cv2.putText(tmp, "warning", (1935, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                cv2.rectangle(tmp, (1920, 110), (2070, 730), color_w, -1)

                y_values.append(prob*200)
                del y_values[0]
                cv2.rectangle(tmp, (1340, 520), (1880, 710), (255, 255, 255), -1)
                for i in range (0,19) :
                    
                    cv2.circle(tmp, (1360 + (i * 27), 700 - int(y_values[i])), radius= 4, color = (255, 0, 0), thickness = -1)
                    cv2.putText(tmp, "Graph", (1550, 480), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                    cv2.putText(tmp, "0", (1320, 700), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 255), 2)
                    cv2.putText(tmp, "0.5", (1312, 615), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 255), 2)
                    cv2.putText(tmp, "1", (1320, 540), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 255), 2)
                        
    # Open two windows
    
    ## Display image ("Emotion Recognition")
    
    ## Display probabilities of emotion
    cv2.imshow('Emotion Recognition', tmp)

    
    # q to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clear program and close windows
camera.release()
cv2.destroyAllWindows()
